![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_RoBertaForTokenClassification.ipynb)

## Import ONNX RoBertaForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `RoBertaForTokenClassification` is only available since in `Spark NLP 5.1.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import RoBERTa models trained/fine-tuned for token classification via `RobertaForTokenClassification` or `TFRobertaForTokenClassification`. These models are usually under `Token Classification` category and have `roberta` in their labels
- Reference: [TFRobertaForTokenClassification](https://huggingface.co/transformers/model_doc/roberta.html#tfrobertafortokenclassification)
- Some [example models](https://huggingface.co/models?filter=roberta&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [philschmid/distilroberta-base-ner-wikiann-conll2003-3-class](https://huggingface.co/philschmid/distilroberta-base-ner-wikiann-conll2003-3-class) model from HuggingFace as an example and load it as a `ORTModelForTokenClassification`, representing an ONNX model.
- In addition to the RoBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [4]:
from transformers import RobertaTokenizer
from optimum.onnxruntime import ORTModelForTokenClassification

MODEL_NAME = 'w11wo/indonesian-roberta-base-posp-tagger'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
No ONNX files were found for w11wo/indonesian-roberta-base-posp-tagger, setting `export=True` to convert the model to ONNX. Don't forget to save the resulting model with `.save_pretrained()`


('onnx_models/w11wo/indonesian-roberta-base-posp-tagger/tokenizer_config.json',
 'onnx_models/w11wo/indonesian-roberta-base-posp-tagger/special_tokens_map.json',
 'onnx_models/w11wo/indonesian-roberta-base-posp-tagger/vocab.json',
 'onnx_models/w11wo/indonesian-roberta-base-posp-tagger/merges.txt',
 'onnx_models/w11wo/indonesian-roberta-base-posp-tagger/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -l {ONNX_MODEL}

total 486440
-rw-r--r-- 1 root root      1639 Jun 16 17:08 config.json
-rw-r--r-- 1 root root    466674 Jun 16 17:08 merges.txt
-rw-r--r-- 1 root root 496619008 Jun 16 17:08 model.onnx
-rw-r--r-- 1 root root       964 Jun 16 17:08 special_tokens_map.json
-rw-r--r-- 1 root root      1272 Jun 16 17:08 tokenizer_config.json
-rw-r--r-- 1 root root   1009470 Jun 16 17:08 vocab.json


- We need to convert `vocab.json` to a plain `vocab.txt` format, as required by Spark NLP.
- Move both `vocab.txt` and `merges.txt` into the assets folder.
- Additionally, we need to extract label-to-ID mappings from the model config and save them as `labels.txt` in the same folder for Spark NLP to use during inference.

In [6]:
import json

!mkdir -p {ONNX_MODEL}/assets && mv {ONNX_MODEL}/merges.txt {ONNX_MODEL}/assets/

with open(f"{ONNX_MODEL}/vocab.json") as f, open(f"{ONNX_MODEL}/assets/vocab.txt", "w") as out:
    out.write("\n".join(json.load(f)))

with open(f"{ONNX_MODEL}/assets/labels.txt", "w") as f:
    f.write("\n".join(ort_model.config.id2label[k] for k in sorted(ort_model.config.id2label)))

In [7]:
!ls -lR {ONNX_MODEL}

onnx_models/w11wo/indonesian-roberta-base-posp-tagger:
total 485988
drwxr-xr-x 2 root root      4096 Jun 16 17:09 assets
-rw-r--r-- 1 root root      1639 Jun 16 17:08 config.json
-rw-r--r-- 1 root root 496619008 Jun 16 17:08 model.onnx
-rw-r--r-- 1 root root       964 Jun 16 17:08 special_tokens_map.json
-rw-r--r-- 1 root root      1272 Jun 16 17:08 tokenizer_config.json
-rw-r--r-- 1 root root   1009470 Jun 16 17:08 vocab.json

onnx_models/w11wo/indonesian-roberta-base-posp-tagger/assets:
total 868
-rw-r--r-- 1 root root    155 Jun 16 17:09 labels.txt
-rw-r--r-- 1 root root 466674 Jun 16 17:08 merges.txt
-rw-r--r-- 1 root root 417357 Jun 16 17:09 vocab.txt


In [8]:
!cat {ONNX_MODEL}/assets/labels.txt

B-PPO
B-KUA
B-ADV
B-PRN
B-VBI
B-PAR
B-VBP
B-NNP
B-UNS
B-VBT
B-VBL
B-NNO
B-ADJ
B-PRR
B-PRK
B-CCN
B-$$$
B-ADK
B-ART
B-CSN
B-NUM
B-SYM
B-INT
B-NEG
B-PRI
B-VBE

Voila! We have our `vocab.txt` and `merges.txt` inside assets directory, along with the extracted labels saved in `labels.txt`.

## Import and Save RoBertaForTokenClassification in Spark NLP


- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [9]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 48.8 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [10]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `RoBertaForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [11]:
from sparknlp.annotator import RoBertaForTokenClassification

tokenClassifier = RoBertaForTokenClassification\
  .loadSavedModel(ONNX_MODEL, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [12]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [13]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your RoBertaForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [14]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 485068
drwxr-xr-x 5 root root      4096 Jun 16 17:12 fields
drwxr-xr-x 2 root root      4096 Jun 16 17:12 metadata
-rw-r--r-- 1 root root 496694922 Jun 16 17:12 roberta_classification_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForTokenClassification model 😊

In [15]:
tokenClassifier_loaded = RoBertaForTokenClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [16]:
tokenClassifier_loaded.getClasses()

['B-NEG',
 'B-VBL',
 'B-ADK',
 'B-PRI',
 'B-$$$',
 'B-PRN',
 'B-VBI',
 'B-NNP',
 'B-ADV',
 'B-UNS',
 'B-ADJ',
 'B-PRR',
 'B-SYM',
 'B-NUM',
 'B-KUA',
 'B-PAR',
 'B-NNO',
 'B-VBP',
 'B-PPO',
 'B-INT',
 'B-CSN',
 'B-VBE',
 'B-VBT',
 'B-PRK',
 'B-CCN',
 'B-ART']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [17]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, NerConverter
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded,
    converter
])

example_text = (
    "Presiden Joko Widodo menghadiri pertemuan penting di Jakarta pada tanggal "
    "17 Agustus 2023, pukul 10:00 WIB, bersama menteri-menteri dan LSM terkait. "
    "Mereka membahas strategi baru untuk mengatasi kemiskinan."
)
data = spark.createDataFrame([[example_text]], ["text"])

result = pipeline.fit(data).transform(data)
result.select("text", "ner.result").show(truncate=False)

result.selectExpr("explode(ner_chunk) as chunk").selectExpr(
    "chunk.result as text",
    "chunk.metadata['entity'] as entity"
).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                          |result                                                                                                                                                                                                                          |
+-----------------------------------------------------------------------------------------------------------------------------------

That's it! You can now go wild and use hundreds of `RoBertaForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
